# Nano GPT

In [3]:
# Download the data file if it doesn't exist 
import urllib.request
import os.path

file_name = "input.txt"
file_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"

if not os.path.isfile(file_name):
    urllib.request.urlretrieve(file_url, file_name)
else:
    print("data file already present")

data file already present


In [4]:
# Read the data file
with open(file_name, "r", encoding="utf-8") as f:
    text = f.read()

In [6]:
# Examine the file contents
print("Length of the dataset in characters: ", len(text))

Length of the dataset in characters:  1115394


In [7]:
# Examine the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.


In [9]:
# set() takes in an iterable sequence and returns all non-repeating elements from the iterable sequence.
chars_set = set(text)
chars_list = list(chars_set)

chars_list

['p',
 "'",
 'K',
 'X',
 'r',
 'R',
 'P',
 'G',
 'D',
 ' ',
 'e',
 'm',
 'L',
 's',
 'T',
 '!',
 't',
 'a',
 'O',
 'C',
 'c',
 'B',
 'l',
 'j',
 'u',
 'M',
 'o',
 'Y',
 'v',
 'i',
 'F',
 'n',
 'h',
 'x',
 'N',
 'V',
 '3',
 'z',
 'U',
 'k',
 'g',
 '?',
 '$',
 ':',
 'W',
 '\n',
 'b',
 'q',
 'I',
 ';',
 'A',
 'E',
 'J',
 'Q',
 'Z',
 '.',
 'd',
 'f',
 'y',
 'S',
 '-',
 ',',
 '&',
 'w',
 'H']